In [122]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
import pandas as pd
# Create the client instance
# client = Elasticsearch("http://localhost:9200")

# # Successful response!
# client.info()

ELASTIC_PASSWORD = "nqWXecniTzykQWUfZpxPliLq"

# Found in the 'Manage Deployment' page
CLOUD_ID = "Recipes-chefWnos:ZXVyb3BlLXdlc3QzLmdjcC5jbG91ZC5lcy5pbzo0NDMkZmYwOTc5MjM4MjhmNDdhNjk4NTZmYzg2M2IzZGNhYTMkNzMwZmZkOGRjZDgxNDRhYzkwMDI1NzM0OTA0ZTM0ZGQ="

# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    http_auth=("elastic", ELASTIC_PASSWORD)
)

# Successful response!
client.info()

{'name': 'instance-0000000000',
 'cluster_name': 'ff097923828f47a69856fc863b3dcaa3',
 'cluster_uuid': 'pxX7iuB_TNOB7WKUuYai4A',
 'version': {'number': '8.8.0',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f',
  'build_date': '2023-05-23T17:16:07.179039820Z',
  'build_snapshot': False,
  'lucene_version': '9.6.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [123]:
recipes = pd.read_json('data/recipes_cleaned_1.json')

In [124]:
for i, data in recipes.iterrows():
    ing = []
    for j in data['ingredients']:
        ing.append(j['ingredient'])
    data['ingredients'] = ing


In [16]:
recipes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15911 entries, 0 to 15910
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         15911 non-null  object
 1   ingredients  15911 non-null  object
 2   steps        15911 non-null  object
 3   description  15911 non-null  object
 4   time         15911 non-null  object
 5   tags         15911 non-null  object
 6   cuisine      15911 non-null  object
dtypes: object(7)
memory usage: 994.4+ KB


In [125]:
def create_index(client, index_name):
    # Define the index settings and mappings
    if client.indices.exists(index_name):
        pass
    index_settings = {
        "settings": {
            "analysis": {
                "filter": {
                    "arabic_stemmer": {
                        "type": "stemmer",
                        "language": "arabic"
                    },
                    "arabic_stop": {
                        "type": "stop",
                        "stopwords": "_arabic_"
                    }
                },
                "analyzer": {
                    "rebuilt_arabic": {
                        "type": "custom",
                        "tokenizer": "standard",
                        "filter": ["lowercase", "decimal_digit", "arabic_stop", "arabic_normalization", "arabic_stemmer"]
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "Id": {
                    "type": "long"
                },
                "name": {
                    "type": "text",
                    "analyzer": "rebuilt_arabic",
                    "search_analyzer": "rebuilt_arabic"
                },
                "ingredients": {
                    "type": "text",
                    "analyzer": "rebuilt_arabic",
                    "search_analyzer": "rebuilt_arabic"
                },
                "steps": {
                    "type": "text",
                    "analyzer": "rebuilt_arabic",
                    "search_analyzer": "rebuilt_arabic"
                },
                "tags": {
                    "type": "text",
                    "analyzer": "rebuilt_arabic",
                    "search_analyzer": "rebuilt_arabic"
                }
            }
        }
    }

    # Create the index with the specified settings and mappings
    create_index_response = client.indices.create(index=index_name, body=index_settings)
    return create_index_response
# Call the function to create the index
response = create_index(client=client, index_name="recipes")
print(response)

C:\Users\Fastora\AppData\Local\Temp\ipykernel_16116\4178635385.py:3: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if client.indices.exists(index_name):
C:\Users\Fastora\AppData\Local\Temp\ipykernel_16116\4178635385.py:57: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  create_index_response = client.indices.create(index=index_name, body=index_settings)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'recipes'}


In [126]:
df_recipes = recipes
#join ingredients list to one string
df_recipes['ingredients'] = df_recipes['ingredients'].apply(lambda x: ' '.join(x))
#join tags list to one string
df_recipes['tags'] = df_recipes['tags'].apply(lambda x: ' '.join(x))
#join steps list to one string
df_recipes['steps'] = df_recipes['steps'].apply(lambda x: ' '.join(x))


In [127]:
def add_recipes(client, index, recipes):
    #add to recipes id column from its index
    recipes['Id'] = recipes.index
    # Convert the dataframe to a dictionary
    recipes_dict = recipes.to_dict("records")
    for recipe in recipes_dict:
        # Index the dictionary of each recipe into Elasticsearch
        resp = client.index(index=index, document=recipe)
        if resp['result'] != 'created':
            print(resp)


In [128]:
add_recipes(client, 'recipes', df_recipes)

In [129]:
def search_by_ingredients(ingredients_query, client, index):
    # Define the search query
    search_query = Q(
        "multi_match",
        fields=["ingredients^2", "name^2", "steps"],
        query=ingredients_query,
        fuzziness="AUTO"
    )

    # Create a search object
    search = client.search(index=index, body={"query": search_query.to_dict()})

    ids = []
    for hit in search["hits"]["hits"]:
        ids.append(hit['_source']['Id'])
        
    return search["hits"]["hits"], ids

In [130]:
q = "بطاطس رز بصل"
result = search_by_ingredients(q, client, 'recipes')
print(result)

C:\Users\Fastora\AppData\Local\Temp\ipykernel_16116\3548199306.py:11: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  search = client.search(index=index, body={"query": search_query.to_dict()})


([{'_index': 'recipes', '_id': 'tgC_WIgBtjLtA0NX8I1S', '_score': 18.764229, '_source': {'name': 'الرز الأصفر بالبصل..', 'ingredients': 'الأرز مرق دجاج ملح البصل الكركم الزيت النباتي الفلفل الأخضر بهارات', 'steps': '1. يسكب الزيت في قدر توضع على نار هادئة، ويضاف له البصل المفروم، ويقلب لدقيقتين، فالأرز الذي يقلب أيضاً لدقيقتين، ثم ينثر الكركم والملح ويوضع مرق الدجاج والبهار. 2. يقلب هذا المزيج، وتغطى القدر، مع التقليب كلّ 5 دقائق حتى يجف الماء تماماً وينضج الأرز. 3. ثم، يضاف بعض الزيت في قدر نظيفة، ويقلى الفلفل الأخضر الحلو حتى يذبل، ثم يخلط بالأرز. 4. \xa0يمكن تزيين هذا الطبق بأوراق البصل الأخصر المقطعة.', 'description': '\n                        ', 'time': '0 دقيقة ', 'tags': 'أطباق الأرز وصفات الأرز أطباق رمضانية', 'cuisine': ' عربي ', 'Id': 4017}}, {'_index': 'recipes', '_id': 'wwC-WIgBtjLtA0NXnIh-', '_score': 15.743984, '_source': {'name': 'بطاطس مقلية مع البصل', 'ingredients': 'البطاطس البصل الزبدة زيت الزيتون بودرة الثوم أعشاب مجففة ملح فلفل أسود', 'steps': '1.\xa0 سخني الزبدة م

In [131]:
ids = result[1]

In [132]:
for i in ids:
    print(df_recipes.loc[df_recipes['Id'] == int(i)]['name'].values[0])

الرز الأصفر بالبصل..
بطاطس مقلية مع البصل
شوربة البطاطس بالبصل
بطاطس حلقات بالبصل
بصل مشوي بالبطاطس
الرز البخاري الأصلي
رز باللحمة والبطاطس والعدس
بطاطس بالتوم والبصل الأخضر
حلقات البصل بالبطاطس والجبن
عجة البيض بالبطاطس والبصل


In [114]:
def search_by_tags(tags_query, client, index):
    # Define the search query
    search_query = Q(
        "multi_match",
        fields=["tags^2", "name", "description"],
        query=tags_query,
        fuzziness="AUTO"
    )

    # Create a search object
    search = client.search(index=index, body={"query": search_query.to_dict()})

    ids = []
    for hit in search["hits"]["hits"]:
        ids.append(hit['_source']['Id'])
        
    return search["hits"]["hits"], ids

In [111]:
def search_by_name(name_query, client, index):
    # Define the search query
    search_query = Q(
        "multi_match",
        fields=["name^2", "ingredients", "steps"],
        query=name_query,
        fuzziness="AUTO"
    )

    # Create a search object
    search = client.search(index=index, body={"query": search_query.to_dict()})

    ids = []
    for hit in search["hits"]["hits"]:
        ids.append(hit['_source']['Id'])
        
    return search["hits"]["hits"], ids

In [118]:
result = search_by_tags('سهلة مكرونة', client, 'recipes')
print(result)

([{'_index': 'recipes', '_id': '4fZRWIgBKtnneZFb9ajM', '_score': 17.12885, '_source': {'name': 'باستا سريعة', 'ingredients': 'البصل طماطم الكزبرة الثوم ملح فلفل أسود الفلفل الأحمر الكمون بهارات ريحان زيت الزيتون الفطر كريمة الطبخ جبن موزاريلا صلصة طماطم', 'steps': 'اسلقي المكرونة مع الزيت والملح. في قدر نضيف الزيت الزيتون ثم نضع البصل وننتظر حتى يذبل. نضيف الثوم ومن ثم الفطر ونخلطهم جيدا ونتركه لدقيقتان لينضج الفطر. ونضيف الطماطم والكزبرة ونحرك\xa0جيدا من ثم نضع صلصة الطماطم ونحركه جيدا ونضع البهارات وننتظر لدقيقتان على نار هادئة. ومن ثم نضيف الريحان وكريمة الطبخ ونغطي الجدر ل5دقائق ومن ثم نضيف جبن الموزريلا. ومن ثم نضيف المعكرونه الذي سلقناها. ونقدمها ساخنة.', 'description': 'باستا سريعة ... وصفة باستا شهية مقدمة من أعضاء مطبخ سيدتي جربوها\xa0', 'time': '30 دقيقة ', 'tags': 'باستا المكرونة مكرونة معكرونة وصفات سهلة وسريعة', 'cuisine': ' إيطالي ', 'Id': 12111}}, {'_index': 'recipes', '_id': 'mvZRWIgBKtnneZFbWYI-', '_score': 17.1098, '_source': {'name': 'شوربة المكرونة بالخضار', 'ingred

C:\Users\Fastora\AppData\Local\Temp\ipykernel_16116\2433964035.py:11: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  search = client.search(index=index, body={"query": search_query.to_dict()})
